# Word Embeddings : le modèle Word2Vec

## Imports

In [12]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [14]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [15]:
infile = f"../data/sents1.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [17]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [18]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [19]:
len(bigram_phrases.vocab.keys())

4310398

Prenons une clé au hasard :

In [20]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [21]:
bigram_phrases.vocab[key_]

42

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [22]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [23]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [24]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [25]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [26]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races', 'indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 

## Entrainement d'un modèle Word2Vec sur ce corpus

In [27]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: user 2min 7s, sys: 493 ms, total: 2min 8s
Wall time: 44.6 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [28]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [29]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [30]:
model.wv["ministre"]

array([-0.07976063,  0.5808709 , -2.8447385 , -0.8138098 ,  2.6350956 ,
        0.27376214,  0.60678893, -0.28601322, -0.06128453,  1.6084018 ,
        0.11835403,  4.305917  ,  3.3926597 ,  0.41320038, -1.305452  ,
       -2.4239078 , -3.1588202 , -2.4033225 , -2.0655422 , -0.3934135 ,
        2.1472776 ,  1.5327018 ,  2.0269358 , -0.9930736 ,  2.9051971 ,
        3.2847638 , -3.1976814 ,  0.51166344,  0.93005764,  4.013298  ,
       -0.60831875, -1.9089836 ], dtype=float32)

### Calculer la similarité entre deux termes

In [31]:
model.wv.similarity("ministre", "roi")

0.59491324

### Chercher les mots les plus proches d'un terme donné

In [32]:
model.wv.most_similar("ministre", topn=10)

[('secretaire', 0.866146981716156),
 ('president', 0.8426625728607178),
 ('vice_-_president', 0.8335685729980469),
 ('directeur', 0.8284115195274353),
 ('gouverneur', 0.8187008500099182),
 ('adjoint', 0.8182088136672974),
 ('au_nom', 0.8151536583900452),
 ('au_ministere', 0.8119288682937622),
 ('membre', 0.8097673654556274),
 ('conseiller', 0.8071581721305847)]

### Faire des recherches complexes à travers l'espace vectoriel

In [33]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))

[('ce_matin', 0.7871467471122742), ('dimanche', 0.7748206257820129), ('berlin', 0.773789644241333), ('nice', 0.7631933689117432), ('marseille', 0.7577680945396423), ('la_monnaie', 0.7552760243415833), ('new_-_york', 0.7516361474990845), ('hier_soir', 0.7411205172538757), ('demain', 0.7401692867279053), ('dimanche_prochain', 0.7346445322036743)]
